In [16]:
# Read data from CSV file
import pandas as pd
transportation_costs_production_to_refinement_df = pd.read_csv('/Users/frieda/Desktop/a1/Cost_Production_to_Refinement.csv')


In [17]:
import gurobipy as gp
from gurobipy import GRB

In [19]:
print(len(transportation_costs_production_to_refinement_df.columns))


3


In [28]:
# Create the transportation_costs_production_to_refinement dictionary
transportation_costs_production_to_refinement = {
    (i, j, cost): 1 for i, j, cost in zip(
        transportation_costs_production_to_refinement_df['ProductionFacility'],
        transportation_costs_production_to_refinement_df['RefinementCenter'],
        transportation_costs_production_to_refinement_df['Cost']
    )
}


In [29]:
# Display the filled dictionary
print(transportation_costs_production_to_refinement)

{(1, 1, 4.252732774): 1, (1, 2, 4.567725519): 1, (1, 3, 4.696484464): 1, (1, 4, 2.67874096): 1, (1, 5, 4.272450694): 1, (2, 1, 2.950472217): 1, (2, 2, 2.636619306): 1, (2, 3, 2.853468713): 1, (2, 4, 5.480026127): 1, (2, 5, 3.037495395): 1, (3, 1, 4.025683134): 1, (3, 2, 5.407799438): 1, (3, 3, 3.380948941): 1, (3, 4, 5.441237127): 1, (3, 5, 4.408862172): 1, (4, 1, 3.46114107): 1, (4, 2, 5.346943887): 1, (4, 3, 4.993663113): 1, (4, 4, 5.96818684): 1, (4, 5, 4.836004068): 1, (5, 1, 5.946814293): 1, (5, 2, 4.749172486): 1, (5, 3, 3.726149815): 1, (5, 4, 3.589158582): 1, (5, 5, 4.937414587): 1, (6, 1, 3.469393658): 1, (6, 2, 2.755014432): 1, (6, 3, 5.601230554): 1, (6, 4, 4.499101123): 1, (6, 5, 4.22405746): 1, (7, 1, 4.729031498): 1, (7, 2, 4.301937659): 1, (7, 3, 3.677403086): 1, (7, 4, 4.158811211): 1, (7, 5, 4.159568059): 1, (8, 1, 5.743809755): 1, (8, 2, 5.571586655): 1, (8, 3, 5.095348142): 1, (8, 4, 5.349864973): 1, (8, 5, 2.625629664): 1, (9, 1, 4.256229818): 1, (9, 2, 5.568655736)

In [30]:
# Data
facilities = range(1, 41)
direct_production_facilities = range(1, 26)
transship_distribution_centers = [1, 2]
transship_production_facilities = range(26, 41)
refinement_centers = range(1, 6)

In [31]:
# Capacity data
direct_production_capacities = {
    1: 462, 2: 103, 3: 460, 4: 325, 5: 227, 6: 217, 7: 205, 8: 521, 9: 548, 10: 191,
    11: 361, 12: 411, 13: 104, 14: 155, 15: 285, 16: 109, 17: 422, 18: 438, 19: 501,
    20: 139, 21: 462, 22: 504, 23: 106, 24: 132, 25: 298
}

transship_distribution_capacities = {
    1: 1317,
    2: 1453
}

transship_production_capacities = {
    1: 374, 2: 444, 3: 395, 4: 245, 5: 378, 6: 408, 7: 435, 8: 175, 9: 415,
    10: 503, 11: 184, 12: 297, 13: 450, 14: 169, 15: 365
}

# Demand data
demand = {
    'Refinery_1': 150, 'Refinery_2': 100, 'Refinery_3': 120, 'Refinery_4': 80, 'Refinery_5': 70
}

# Transportation costs data
transportation_costs_production_to_transshipment = {
    (i, j, k): cost for i, j, k, cost in [
        (1, 1, 2, 2.378825860105162), (1, 2, 2, 0.8638421476510945),
        (2, 1, 1, 1.6669815621992674), (2, 2, 2, 2.1194881741170786),
        (3, 1, 1, 2.1748804139601248), (3, 2, 2, 0.9481837554796164),
        (4, 1, 1, 2.6774155296149242), (4, 2, 2, 0.5924200958027974),
        (5, 1, 1, 2.8202478310597714), (5, 2, 2, 0.9723289185578572),
        (6, 1, 1, 2.8538900451592584), (6, 2, 2, 2.694342703),
        (7, 1, 1, 1.4070281695438536), (7, 2, 2, 1.4283253152681572),
        (8, 1, 1, 2.484999068788313), (8, 2, 2, 0.9534766574967986),
        (9, 1, 1, 1.786372012026526), (9, 2, 2, 0.6366996211997129),
        (10, 1, 1, 0.6885277071594851), (10, 2, 2, 2.2597032761699474),
        (11, 1, 1, 2.4012857907559626), (11, 2, 2, 1.8001066456314834),
        (12, 1, 1, 0.5643501932314339), (12, 2, 2, 2.1248424357447133),
        (13, 1, 1, 1.9860695794538117), (13, 2, 2, 2.3471795283740797),
        (14, 1, 1, 1.6399685751473123), (14, 2, 2, 1.619281027186056),
        (15, 1, 1, 0.6490466268590618), (15, 2, 2, 1.9688116627184051)
    ]
}


transportation_costs_transshipment_to_refinement = {
    (i, j, k): cost for i, j, k, cost in [
        (1, 1, 1, 1.5723285102511508), (1, 1, 2, 3.465473956167708),
        (1, 1, 3, 2.2440622480675154), (1, 1, 4, 3.773839152270214),
        (1, 1, 5, 3.262651798567284), (1, 2, 1, 1.9004245214843398),
        (1, 2, 2, 1.6131231982898153), (1, 2, 3, 1.7072324080263828),
        (1, 2, 4, 2.705395683206701), (1, 2, 5, 2.257428421003834),
    ]
}

In [32]:
# Create the optimization model
model = gp.Model("Can2Oil")

In [33]:

# Decision variables
x = {}
for i in facilities:
    for j in facilities if i <= 25 else transship_distribution_centers:
        x[i, j] = model.addVar(vtype=GRB.CONTINUOUS, name=f"x_{i}_{j}")

In [34]:

# Objective function: Minimize total transportation cost
model.setObjective(
    sum(transportation_costs_production_to_transshipment[(i, j, k)] * x[i, k] for i, j, k in transportation_costs_production_to_transshipment) +
    sum(transportation_costs_transshipment_to_refinement[(i, j, k)] * x[j, k] for i, j, k in transportation_costs_transshipment_to_refinement) +
    sum(transportation_costs_production_to_refinement[(i, j, k)] * x[i, k] for i, j, k in transportation_costs_production_to_refinement),

    GRB.MINIMIZE
)

KeyError: (1, 4.252732774)

In [9]:


# Constraints
# Capacity constraints for direct production facilities
for i in direct_production_facilities:
    model.addConstr(sum(x[i, j] for j in refinement_centers) <= direct_production_capacities[i], f"Capacity_Direct_{i}")

# Capacity constraints for transship distribution centers
for j in transship_distribution_centers:
    model.addConstr(sum(x[i, j] for i in facilities) <= transship_distribution_capacities[j], f"Capacity_Transship_Distribution_{j}")

# Capacity constraints for transship production facilities
for i in transship_production_facilities:
    model.addConstr(sum(x[i, j] for j in transship_distribution_centers) <= transship_production_capacities[i], f"Capacity_Transship_Production_{i}")

# Demand constraints
for j in refinement_centers:
    model.addConstr(sum(x[i, j] for i in facilities) == demand[f"Refinery_{j}"], f"Demand_{j}")

# Solve the model
model.optimize()

# Print the optimal solution
if model.status == GRB.OPTIMAL:
    print("\nOptimal solution:")
    for i, j in x:
        if x[i, j].x > 0:
            print(f"Ship {x[i, j].x} million pounds from Facility {i} to Refinery/Distribution {j}")

    print("\nTotal cost:", model.objVal)
else:
    print("No solution found.")


KeyError: (1, 4.252732774348948)

Can2Oil sources canola oil from 40 different production facilities in Canada (5), United States (5),
Mexico (5), China (5), France (5), India (5), Ukraine (5), Germany (5). While canola oil from Canada,
US, Mexico, China, and France (i.e., facilities indexed from i = 1, ..., 25) can be shipped directly to its
refinement centers, canola oil from India, Ukraine, and Germany (facilities indexed from i = 26, ..., 40)
need to be transshipped through an intermediary that operates two large distribution centers in Italy
(j = 1) and Greece (j = 2). Note that Can2Oil owns five refinement centers in North America.

To determine the optimal sourcing plan that minimizes total costs, the company has provided data on
transportation costs per million pound, production and distribution capacities (in millions of pounds),
and demand (in millions of pounds). Formulate and solve a linear program using this data.